In [3]:
import laspy

with laspy.open('d2RCD-4-grafos-joint.las') as f:
    print(f"Point format:       {f.header.point_format}")
    print(f"Number of points:   {f.header.point_count}")
    print(f"Number of vlrs:     {len(f.header.vlrs)}")

    

Point format:       <PointFormat(1, 0 bytes of extra dims)>
Number of points:   18411307
Number of vlrs:     1


Laspy can actually scale the x, y, and z dimensions for you. Upper case dimensions (las_file.X, las_file.Y, las_file.Z) give the raw integer dimensions, while lower case dimensions (las_file.x, las_file.y, las_file.z) give the scaled value.


In [7]:
import laspy

# Open the LAS file
las_file = laspy.read("d2RCD-4-grafos-joint.las")

# Print information about the LAS file
print("Number of points:", las_file.header.point_count)
#print("Point format:", las_file.header.point_format_id)
print("X range:", las_file.header.min[0], "-", las_file.header.max[0])
print("Y range:", las_file.header.min[1], "-", las_file.header.max[1])
print("Z range:", las_file.header.min[2], "-", las_file.header.max[2])


Number of points: 18411307
X range: 431290.71 - 432412.7
Y range: 4598173.71 - 4599550.71
Z range: 63.14 - 167.26


In [12]:
import argparse
import os
from plyfile import PlyData, PlyElement
import numpy as np
from sklearn.decomposition import PCA
#import pptk
#import open3d as o3d
import laspy

In [19]:
import math
#PARTIR ORIGINAL MIO HECHO ANTES
dataset_folder = 'las/'
for filename in os.listdir(dataset_folder):
    f = os.path.join(dataset_folder, filename)
    if os.path.isfile(f):
        las = laspy.read(f)
        
        point_data = np.stack([las.x, las.y, las.z], axis=0) #.transpose((1, 0))
        clases_a_reducir = np.array(las.classification)


        def borrar_ignorados(arr, valores_borrar, column_index):
            #column_index = 4
            # Especificar el valor que quieres eliminar
            #valores_borrar = [0,15,21,27]

            for value_to_delete in valores_borrar:
                # Crear una máscara booleana para identificar las filas a eliminar
                mask = arr[:, column_index] == value_to_delete
                # Eliminar las filas que cumplen la condición
                arr = np.delete(arr, np.where(mask), axis=0)
            return arr

        xyzic = np.stack([las.x, las.y, las.z, las.intensity, las.classification], axis=0).transpose((1, 0))
        xyz = np.stack([las.x, las.y, las.z], axis=0).transpose((1, 0))


        #nuevos_puntos = borrar_ignorados(xyzic, [0,10,15,21,27], 4)
        nuevos_puntos = xyzic

        all_x = point_data[0]
        all_y = point_data[1]
        all_z = point_data[2]

        x_min = np.memmap.min(all_x)
        x_max = np.memmap.max(all_x)
        y_min = np.memmap.min(all_y)
        y_max = np.memmap.max(all_y)
        z_min = np.memmap.min(all_z)
        z_max = np.memmap.max(all_z)

        len_split = 51  #25000  #the length you want to have in each fragment of new division of pointcloud

        divide_x_in = math.floor((x_max-x_min) / len_split) #cogemos el de arriba para asegurar tener todo. quiza los trozos no son de 50x50 sino 49.8x49.9ç
        divide_y_in = math.floor((y_max-y_min) / len_split) #cogemos el de arriba para asegurar tener todo. quiza los trozos no son de 50x50 sino 49.8x49.9ç

        print(divide_x_in)
        print(divide_y_in)


    
        x_grid = np.linspace(x_min, x_max, divide_x_in + 1)
        y_grid = np.linspace(y_min, y_max, divide_y_in + 1)

        save_dir = "cortados/"
        #filename = "Aerolaser"



        count = 0

        for i in range(len(x_grid)-1):
            for j in range(len(y_grid)-1):
                if count <= len(y_grid)-1: 
                    mask_x = np.logical_and(nuevos_puntos[:,0] >= x_grid[i], nuevos_puntos[:,0] <= x_grid[i+1])
                else: 
                    mask_x = np.logical_and(nuevos_puntos[:,0] >= x_grid[i], nuevos_puntos[:,0] < x_grid[i+1])
                count += 1
                
                filtered_x = nuevos_puntos[mask_x]
                
                mask_y = np.logical_and(filtered_x[:,1] >= y_grid[j], filtered_x[:,1] < y_grid[j+1])
                filtered_y = filtered_x[mask_y]

                #v = pptk.viewer(filtered_x)
                #v = pptk.viewer(filtered_y)
                if(len(filtered_y)>= 4096):

                    np.save(os.path.join(save_dir, os.path.splitext(filename)[0]+f"_{str(count).zfill(3)}"), filtered_y)



21
27


In [120]:
import open3d as o3d
import numpy as np
from matplotlib import cm


numpointstoreduce = 16384

# Define colors for each class
MisColores = {
    1: np.array([0.5,0.25,0]), #terreno
    2: np.array([0.1,0.6,0]),  #vegetación
    3: np.array([0.99,0,0]), #coche
    4: np.array([0.5,0.5,0.5]), #torre
    5: np.array([0.99,0,0.99]), #cable
    6: np.array([0.1,0.1,0.1]), #valla/muro
    7: np.array([0.99,0.99,0]), #farola
    8: np.array([0.15,0.09,0.9]), #edificio
}

#data = np.load('pruebas_las_cortados/section_431749.71_4598734.71.npy')

data = np.load('cortados/d2RCD-4-grafos-joint_488.npy')
#data = np.load('nube_de_puntos.npy')
x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
c = data[:, 4].astype(int)
r = data[:, 3]

points = data[:, 0:3]

# Assign colors to each point based on the corresponding value in the c array
colors = np.zeros_like(points)
for i, value in MisColores.items():
    mask = (c == i)
    colors[mask] = value

cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)
cloud.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([cloud])

if len(cloud.points) > numpointstoreduce:
    print("Downsample the point cloud with a voxel of 0.05")
    downpcd = cloud.farthest_point_down_sample(num_samples=numpointstoreduce)
    o3d.visualization.draw_geometries([downpcd])
    len(downpcd.points)

In [121]:
#SPLIT VHATGPT TIENE BUIENA PUNTA
import laspy
import numpy as np

# Open the LAS file
las_file = laspy.read("las/d2RCD-4-grafos-joint.las")

# Get the spatial extent of the LAS file
min_x, min_y, min_z = np.min(las_file.x), np.min(las_file.y), np.min(las_file.z)
max_x, max_y, max_z = np.max(las_file.x), np.max(las_file.y), np.max(las_file.z)
extent = np.array([min_x, min_y, max_x, max_y])

# Define the size of the smaller sections
section_size = 51.0  # meters

# Loop over the smaller sections
for x_min in np.arange(min_x, max_x, section_size):
    for y_min in np.arange(min_y, max_y, section_size):
        # Define the bounding box for this section
        x_max, y_max = x_min + section_size, y_min + section_size
        section_extent = np.array([x_min, y_min, x_max, y_max])

        # Find the points that are inside the section
        section_indices = np.where((las_file.x >= x_min) & (las_file.x < x_max) & 
                                   (las_file.y >= y_min) & (las_file.y < y_max))[0]

        # If there are points inside the section, save a new LAS file
        if len(section_indices) > 0:
            section = las_file[section_indices]
            section_filename = f"section_{x_min:.2f}_{y_min:.2f}.las"
            section.header.offset = section_extent[:3]
            section.header.scale = las_file.header.scale
            section.header.min = section_extent[:3]
            section.header.max = section_extent[2:]
            section.write(section_filename)


KeyboardInterrupt: 

In [93]:
import open3d as o3d

data = np.load('cortados/d2RCD-4-grafos-joint_024.npy')

x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
r = data[:, 3]
c = data[:, 4]

points = data[:,0:3]

"""
if colors is not None:
    if isinstance(colors, np.ndarray):
        cloud.colors = o3d.utility.Vector3dVector(colors)
    else: cloud.paint_uniform_color(colors)
"""
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points) 
o3d.visualization.draw_geometries([cloud])

In [47]:
import open3d as o3d
import numpy as np
from matplotlib import cm

data = np.load('pruebas_las_cortados/section_431749.71_4598734.71.npy')

x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
c = data[:, 3]
r = data[:, 4]

points = data[:, 0:3]

cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([cloud])


Cambiar las por npy


In [42]:
import os
import laspy
import numpy as np

# Directory containing LAS files
las_dir = 'pruebas_las_cortados/'

# Loop over files in directory
for filename in os.listdir(las_dir):
    if filename.endswith('.las'):
        # Load LAS file
        las_file = laspy.read(os.path.join(las_dir, filename))

        # Extract x, y, z, and intensity values from LAS file
        x = las_file.x
        y = las_file.y
        z = las_file.z
        c = las_file.classification
        intensity = las_file.intensity

        # Combine x, y, z, and intensity values into a single numpy array
        data = np.column_stack((x, y, z, c, intensity))

        # Save numpy array to NPY file
        np.save(os.path.join(las_dir, filename.replace('.las', '.npy')), data)



In [116]:
import open3d as o3d
import numpy as np
from matplotlib import cm

# Define colors for each class
MisColores = {
    1: np.array([0.5,0.25,0]), #terreno
    2: np.array([0.1,0.6,0]),  #vegetación
    3: np.array([0.99,0,0]), #coche
    4: np.array([0.5,0.5,0.5]), #torre
    5: np.array([0.99,0,0.99]), #cable
    6: np.array([0.1,0.1,0.1]), #valla/muro
    7: np.array([0.99,0.99,0]), #farola
    8: np.array([0.15,0.09,0.9]), #edificio
}

#data = np.load('pruebas_las_cortados/section_431749.71_4598734.71.npy')
data = np.load('nube_de_puntos.npy')
x = data[:, 0]
y = data[:, 1]
z = data[:, 2]
c = data[:, 4].astype(int)
r = data[:, 3]

points = data[:, 0:3]

# Assign colors to each point based on the corresponding value in the c array
colors = np.zeros_like(points)
for i, value in MisColores.items():
    mask = (c == i)
    colors[mask] = value

cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)
cloud.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([cloud])

las_dir = 'cortados/'

"""
for filename in os.listdir(las_dir):
    if filename.endswith('.npy'):
        data = np.load(las_dir+filename)
        x = data[:, 0]
        y = data[:, 1]
        z = data[:, 2]
        c = data[:, 4].astype(int)
        r = data[:, 3]

        points = data[:, 0:3]

        # Assign colors to each point based on the corresponding value in the c array
        colors = np.zeros_like(points)
        for i, value in MisColores.items():
            mask = (c == i)
            colors[mask] = value

        cloud = o3d.geometry.PointCloud()
        cloud.points = o3d.utility.Vector3dVector(points)
        cloud.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([cloud])
"""


"\nfor filename in os.listdir(las_dir):\n    if filename.endswith('.npy'):\n        data = np.load(las_dir+filename)\n        x = data[:, 0]\n        y = data[:, 1]\n        z = data[:, 2]\n        c = data[:, 4].astype(int)\n        r = data[:, 3]\n\n        points = data[:, 0:3]\n\n        # Assign colors to each point based on the corresponding value in the c array\n        colors = np.zeros_like(points)\n        for i, value in MisColores.items():\n            mask = (c == i)\n            colors[mask] = value\n\n        cloud = o3d.geometry.PointCloud()\n        cloud.points = o3d.utility.Vector3dVector(points)\n        cloud.colors = o3d.utility.Vector3dVector(colors)\n        o3d.visualization.draw_geometries([cloud])\n"

Reunir de nuevo el dataset

In [114]:
las_dir = 'cortados/'

datos_fragmentos = []

for filename in os.listdir(las_dir):
    if filename.endswith('.npy'):
        #data = np.load(las_dir+filename)
        datos_fragmentos.append(np.load(las_dir+filename))

# concatenamos todos los datos de los fragmentos en una única matriz
nube_de_puntos = np.vstack(datos_fragmentos)

# guardamos la nube de puntos resultante como un archivo .npy
np.save("nube_de_puntos_reconstruida.npy", nube_de_puntos)


    